In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE178228"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE178228"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE178228.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE178228.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE178228.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Ixekizumab treatment of patients with moderate-to-severe plaque psoriasis."
!Series_summary	"Objectives: To asses skin clearance and patient-reported outcomes for ixekizumab treatment. Methods: IXORA-R enrolled adults with moderate-to-severe plaque psoriasis, defined as static Physician’s Global Assessment ≥ 3, PASI ≥ 12 and involved body surface area ≥ 10%. The trial was registered with ClinicalTrials.gov (NCT03573323)."
!Series_overall_design	"Eligible patients were ≥ 18 years old with chronic plaque psoriasis with a static Physician’s Global Assessment of Disease (sPGA) score of ≥ 3 (moderate), a Psoriasis Area and Severity Index (PASI) ≥ 12, and ≥ 10% body surface area involvement at screening and baseline. Psoriatic plaque skin samples were collected at baseline, week 1, week2, and week 4 after ixekizumab treatment initiation."
Sample Characteristics Dictionary:
{0: ['visitid: 2', 'visitid: 4', 'visitid: 3', 'visitid: 5'], 1: ['treatment: Ixe

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the backgrounds, this appears to be a gene expression dataset since it involves measuring gene expression data at different time points after ixekizumab treatment
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait, key 2 contains PASI scores which indicate psoriasis severity
trait_row = 2

# Unfortunately, age and gender data are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert PASI score to a continuous value"""
    try:
        # Extract the value after the colon and convert to float
        if ':' in value:
            val = value.split(':', 1)[1].strip()
            return float(val)
        return None
    except:
        return None

def convert_age(value):
    """Convert age value to a continuous value (not used as age is not available)"""
    return None

def convert_gender(value):
    """Convert gender to binary (not used as gender is not available)"""
    return None

# 3. Save Metadata
# Trait data availability is True if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort,
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using geo_select_clinical_features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,  # Using the input dataframe from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of clinical features:")
    print(preview_df(clinical_features))
    
    # Save the clinical features to the specified output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM5384796': [14.7], 'GSM5384797': [2.9], 'GSM5384798': [7.2], 'GSM5384799': [15.5], 'GSM5384800': [3.0], 'GSM5384801': [8.2], 'GSM5384802': [6.4], 'GSM5384803': [3.0], 'GSM5384804': [6.0], 'GSM5384805': [7.8], 'GSM5384806': [10.3], 'GSM5384807': [20.1], 'GSM5384808': [3.2], 'GSM5384809': [5.0], 'GSM5384810': [8.39999999999999], 'GSM5384811': [2.6], 'GSM5384812': [22.8], 'GSM5384813': [4.7], 'GSM5384814': [13.4], 'GSM5384815': [4.3], 'GSM5384816': [10.0], 'GSM5384817': [10.6], 'GSM5384818': [7.2], 'GSM5384819': [1.6], 'GSM5384820': [10.2], 'GSM5384821': [4.4], 'GSM5384822': [9.2], 'GSM5384823': [12.6], 'GSM5384824': [12.0], 'GSM5384825': [5.8], 'GSM5384826': [5.0], 'GSM5384827': [11.9], 'GSM5384828': [12.0], 'GSM5384829': [3.0], 'GSM5384830': [12.7], 'GSM5384831': [0.7], 'GSM5384832': [4.1], 'GSM5384833': [1.8], 'GSM5384834': [5.2], 'GSM5384835': [2.7], 'GSM5384836': [13.2], 'GSM5384837': [15.2], 'GSM5384838': [13.8999999999999], 'GSM5384839': [4.3], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')

Gene data dimensions: 70523 genes × 117 samples


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# The identifiers have the format "XXXXXXX_st" which appears to be probe IDs from a microarray
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They need to be mapped to proper gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Inspect the SOFT file structure to understand the annotation format
# Read the first few lines of the SOFT file to examine its structure
import gzip
print("Preview of SOFT file content:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 20:  # Print first 20 lines to understand structure
            break
print("...\n")

# 3. Try different approaches to extract gene annotation data
# First, let's try the default method to see what's actually in the file
gene_annotation = get_gene_annotation(soft_file)
print("Gene annotation dataframe using default method:")
print(f"Shape: {gene_annotation.shape}")
print(f"Columns: {gene_annotation.columns.tolist()}")
print(gene_annotation.head(3))

# 4. Check if there's another section in the file that might contain the mapping
# Look for platform annotation information in the SOFT file
print("\nSearching for platform annotation section in SOFT file...")
with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    capture = False
    for i, line in enumerate(f):
        if "^PLATFORM" in line:
            capture = True
            platform_lines.append(line.strip())
        elif capture and line.startswith("!platform_table_begin"):
            platform_lines.append(line.strip())
            for j in range(10):  # Capture the next 10 lines to understand the table structure
                try:
                    platform_line = next(f).strip()
                    platform_lines.append(platform_line)
                except StopIteration:
                    break
            break
    
    print("\n".join(platform_lines))

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Preview of SOFT file content:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE178228
!Series_title = Ixekizumab treatment of patients with moderate-to-severe plaque psoriasis.
!Series_geo_accession = GSE178228
!Series_status = Public on Apr 10 2023
!Series_submission_date = Jun 15 2021
!Series_last_update_date = Apr 11 2023
!Series_pubmed_id = 36967086
!Series_summary = Objectives: To asses skin clearance and patient-reported outcomes for ixekizumab treatment. Methods: IXORA-R enrolled adults with moderate-to-severe plaque psoriasis, defined as static Physician’s Global Assessment ≥ 3, PASI ≥ 12 and involved body surface area ≥ 10%. The trial was registered with ClinicalTrials.gov (NCT03573323).
!Series_overall_design = Eligible patients were ≥ 18 years old with chronic plaque psoriasis with a static Physician’s Global Assessmen

Gene annotation dataframe using default method:
Shape: (8322061, 15)
Columns: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']
                ID      probeset_id seqname strand  start   stop  \
0  TC01000001.hg.1  TC01000001.hg.1    chr1      +  11869  14409   
1  TC01000002.hg.1  TC01000002.hg.1    chr1      +  29554  31109   
2  TC01000003.hg.1  TC01000003.hg.1    chr1      +  69091  70008   

   total_probes                                    gene_assignment  \
0          49.0  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...   
1          60.0  ENST00000408384 // MIR1302-11 // microRNA 1302...   
2          30.0  NM_001005484 // OR4F5 // olfactory receptor, f...   

                                     mrna_assignment  \
0  NR_046018 // RefSeq // Homo sapiens DEAD/H (As...   
1  ENST00000408384 // ENSEMBL // ncrna:miRNA chro...   
2  NM_001005484 

### Step 6: Gene Identifier Mapping

In [7]:
# First, let's get more information about the dataset platform from the SOFT file
soft_platform_info = None
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith('!Series_platform_id'):
            soft_platform_info = line.strip()
            break

print(f"Platform information: {soft_platform_info}")

# Check for matching probe IDs between expression data and annotation
common_ids = set(gene_data.index) & set(gene_annotation['ID'])
print(f"Number of common probe IDs between expression data and annotation: {len(common_ids)}")

# Since we have matches, let's filter the annotation to only include these probes
filtered_annotation = gene_annotation[gene_annotation['ID'].isin(gene_data.index)].copy()

# Create mapping dataframe with probe IDs and gene symbols extracted from gene_assignment
mapping_df = filtered_annotation[['ID', 'gene_assignment']].copy()
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})

# Display a sample of the mapping dataframe before processing
print("\nSample of mapping dataframe before extraction:")
print(mapping_df.head(2))

# Apply the extract_human_gene_symbols function to extract gene symbols
mapping_df['Gene'] = mapping_df['Gene'].apply(extract_human_gene_symbols)

# Check if we got any gene symbols
sample_with_genes = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)].head(5)
print("\nSample of mapping after extracting gene symbols:")
print(sample_with_genes)

# Apply the gene mapping to convert probe-level measurements to gene expression
print("\nConverting probe measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check the shape of the resulting gene expression dataframe
print(f"Gene expression dataframe shape: {gene_data.shape}")
print("Sample of gene expression data:")
print(gene_data.head(3) if not gene_data.empty else "Empty gene data frame")

# If we have extracted gene symbols correctly, normalize them
if not gene_data.empty:
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"\nFinal gene expression data shape after normalization: {gene_data.shape}")
    print("Sample of normalized gene expression data:")
    print(gene_data.head(3))
else:
    print("\nWarning: Gene expression data is empty after mapping.")
    # If mapping failed, use the original probe IDs
    print("Using original probe IDs as fallback...")
    gene_data = get_genetic_data(matrix_file)
    
    # Create a note about the mapping issue
    mapping_note = ("Gene mapping resulted in empty data. Using original probe IDs "
                   "which may affect interpretability of results.")
    print(f"\nNOTE: {mapping_note}")

# Save gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Confirm gene data is still available
is_gene_available = gene_data.shape[0] > 0
print(f"Gene data available: {is_gene_available} (with {gene_data.shape[0]} features)")


Platform information: !Series_platform_id = GPL17586


Number of common probe IDs between expression data and annotation: 70523



Sample of mapping dataframe before extraction:
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...



Sample of mapping after extracting gene symbols:
                ID                                               Gene
0  TC01000001.hg.1                           [DDX11L1, DEAD, DDX11L5]
1  TC01000002.hg.1  [MIR1302-11, MIR1302-10, MIR1302-9, MIR1302-2,...
2  TC01000003.hg.1                                      [OR4F5, NULL]
3  TC01000004.hg.1                                 [NULL, RP11-34P13]
4  TC01000005.hg.1                                 [NULL, RP4-669L17]

Converting probe measurements to gene expression data...


Gene expression dataframe shape: (0, 117)
Sample of gene expression data:
Empty gene data frame

Using original probe IDs as fallback...



NOTE: Gene mapping resulted in empty data. Using original probe IDs which may affect interpretability of results.


Gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE178228.csv
Gene data available: True (with 70523 features)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 0 genes × 117 samples
Normalized gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE178228.csv
Extracting clinical features from the original source...


Extracted clinical features preview:
{'GSM5384796': [14.7], 'GSM5384797': [2.9], 'GSM5384798': [7.2], 'GSM5384799': [15.5], 'GSM5384800': [3.0], 'GSM5384801': [8.2], 'GSM5384802': [6.4], 'GSM5384803': [3.0], 'GSM5384804': [6.0], 'GSM5384805': [7.8], 'GSM5384806': [10.3], 'GSM5384807': [20.1], 'GSM5384808': [3.2], 'GSM5384809': [5.0], 'GSM5384810': [8.39999999999999], 'GSM5384811': [2.6], 'GSM5384812': [22.8], 'GSM5384813': [4.7], 'GSM5384814': [13.4], 'GSM5384815': [4.3], 'GSM5384816': [10.0], 'GSM5384817': [10.6], 'GSM5384818': [7.2], 'GSM5384819': [1.6], 'GSM5384820': [10.2], 'GSM5384821': [4.4], 'GSM5384822': [9.2], 'GSM5384823': [12.6], 'GSM5384824': [12.0], 'GSM5384825': [5.8], 'GSM5384826': [5.0], 'GSM5384827': [11.9], 'GSM5384828': [12.0], 'GSM5384829': [3.0], 'GSM5384830': [12.7], 'GSM5384831': [0.7], 'GSM5384832': [4.1], 'GSM5384833': [1.8], 'GSM5384834': [5.2], 'GSM5384835': [2.7], 'GSM5384836': [13.2], 'GSM5384837': [15.2], 'GSM5384838': [13.8999999999999], 'GSM5384839': [4.